# Google Code Jam 2017 — Qualification Round — problem D
## User: jdemeyer

This is a Jupyter notebook to be run with SageMath version 8.0.beta1 on a 64-bit GNU/Linux system. Although the precise version of SageMath probably does not matter that much.

In [47]:
%%cython

import os, sys, datetime, time, collections
from copy import copy
from sage.rings.integer cimport Integer


def log(msg):
    sys.stderr.write(msg + "\n")
    sys.stderr.flush()


class CodejamProblem(object):
    def __init__(self, input):
        self.inputlines = iter(input.splitlines())
        self.cases = []
    
    def readline(self):
        return next(self.inputlines)
        
    def readint(self):
        return Integer(self.readline())

    def readints(self):
        return [Integer(x) for x in self.readline().split()]
        
    def solve(self, f=sys.stdout, raw=False):
        for i, case in enumerate(self.cases, 1):
            sig_check()
            ans = self.solvecase(case)
            if raw:
                ans = repr(ans)
            else:
                ans = self.formatanswer(ans)
            f.write("Case #{0}: {1}\n".format(i, ans.strip()))
        f.flush()
        
    def solvecheck(self, output):
        from StringIO import StringIO
        out = StringIO()
        self.solve(out)
        assert out.getvalue() == output
            
    def formatanswer(self, ans):
        if isinstance(ans, (tuple, list)):
            return " ".join(str(x) for x in ans)
        else:
            return str(ans)

    @classmethod
    def precompute(cls):
        pass
    
    @classmethod
    def autosolve(cls, filein, fileout, *args, **kwds):
        log("precomputing...")
        cls.precompute()

        log("autosolving...")

        nexc = 0
        while nexc < 10:
            sig_check()
            t0 = datetime.datetime.now()
            try:
                input = open(filein).read()
            except IOError:
                time.sleep(0.2)
                continue
            d = datetime.datetime.now() - t0
            log("Read input in %.2fs" % d.total_seconds())
            
            t0 = datetime.datetime.now()
            try:
                problem = cls(input, *args, **kwds)
            except Exception:
                from traceback import print_exc
                print_exc(file=sys.stderr)
                nexc += 1
                time.sleep(0.5)
                continue
            d = datetime.datetime.now() - t0
            ncases = len(problem.cases)
            log("Parsed input in %.2fs, got %s cases" % (d.total_seconds(), ncases))
            
            t0 = datetime.datetime.now()
            with open(fileout, 'w') as out:
                problem.solve(out)
            d = datetime.datetime.now() - t0
            log("Solved problem in %.2fs" % d.total_seconds())

            problem.notify()
            return
        
    @staticmethod
    def notify():
        os.system("mplayer /usr/share/apps/kgoldrunner/themes/default/victory.ogg >/dev/null")

        
from sage.all import Matrix, ZZ

class Problem(CodejamProblem):
    def __init__(self, input):
        CodejamProblem.__init__(self, input)
        
        T = self.readint()
        for i in range(T):
            N, M = self.readints()
            case = [["." for _ in range(N)] for _ in range(N)]
            for k in range(M):
                model, R, C = self.readline().split()
                case[int(R)-1][int(C)-1] = model
            self.cases.append(case)

    def solvecase(self, case):
        n = len(case)
        
        orthomat = self.solve_ortho(case)
        diagmat = self.solve_diag(case, 0)
        diagmat += self.solve_diag(case, 1)
        
        points = 0
        changes = []
        for i in range(n):
            for j in range(n):
                if orthomat[i,j]:
                    if diagmat[i,j]:
                        c = "o"
                        points += 2
                    else:
                        c = "x"
                        points += 1
                else:
                    if diagmat[i,j]:
                        c = "+"
                        points += 1
                    else:
                        c = "."
                        points += 0
                if c != case[i][j]:
                    changes.append((c,i+1,j+1))
        return points, changes

    def formatanswer(self, ans):
        points, changes = ans
        ret = "%i %i\n" % (points, len(changes))
        for ch in changes:
            ret += "%s %i %i\n" % ch
        return ret
        
    def solve_ortho(self, case):
        n = len(case)
        M = Matrix(ZZ, n, n)
        freerows = set(range(n))
        freecols = set(range(n))
        
        for i in range(n):
            for j in range(n):
                if case[i][j] in "xo":
                    M[i,j] = 1
                    freerows.remove(i)
                    freecols.remove(j)
                    
        freerows = sorted(freerows)
        freecols = sorted(freecols)
                    
        while freerows:
            i = freerows.pop()
            j = freecols.pop()
            M[i,j] = 1
            
        return M
    
    def solve_diag(self, case, parity):
        n = len(case)
        M = Matrix(ZZ, n, n)
        freeplus = set(k for k in range(2*n-1) if k%2 == parity)
        freemin = set(k for k in range(n) if k%2 == parity)
        freemin |= set(-k for k in range(n) if k%2 == parity)
        
        for i in range(n):
            for j in range(n):
                if (i+j) % 2 == parity and case[i][j] in "+o":
                    M[i,j] = 1
                    freeplus.remove(i+j)
                    freemin.remove(i-j)
                    
        freeplus = sorted(freeplus, key=lambda x: abs(x-(n-1)))
        freemin = sorted(freemin, key=lambda x: abs(x))

        while freeplus:
            p = freeplus.pop()
            minbound = min(p, 2*(n-1)-p)
            i = len(freemin) - 1
            while i >= 0 and not abs(freemin[i]) <= minbound:
                i -= 1
            if i < 0:
                continue
            m = freemin.pop(i)
            i = (p + m) // 2
            j = (p - m) // 2
            M[i,j] = 1
            
        return M

In [48]:
input="""
3
2 0
1 1
o 1 1
3 4
+ 2 3
+ 2 1
x 3 1
+ 2 2
"""

output="""
Case #1: 4 3
o 2 2
+ 2 1
x 1 1
Case #2: 2 0
Case #3: 6 2
o 2 3
x 1 2
"""

In [49]:
input = "".join(line+"\n" for line in input.splitlines() if line)
output = "".join(line+"\n" for line in output.splitlines() if line)

In [50]:
P = Problem(input)

In [52]:
P.solve(raw=True)
P.solve()
#P.solvecheck(output)

Case #1: (4, [('x', 1, 1), ('+', 1, 2), ('o', 2, 2)])
Case #2: (2, [])
Case #3: (6, [('x', 1, 2), ('o', 2, 3)])
Case #1: 4 3
x 1 1
+ 1 2
o 2 2
Case #2: 2 0
Case #3: 6 2
x 1 2
o 2 3


In [ ]:
P.autosolve("in/D-small-attempt0.in", "out")

precomputing...
autosolving...
